In [12]:
import pandas as pd 
import numpy as np
import re
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 

In [13]:
comments=pd.read_excel('com.xlsx', names=['question','label'], header=None )

In [14]:
comments

,question,label
0,Добрый день. А как можно заказать?,0
1,Цена пожалуйста,2
2,Маска сколько стоит?,2
3,S размер есть,1
4,45 размер есть?,1
...,...,...
238,Вы отправляете в др города?,0
239,"Есть в начиии в магазине S,M?",1
240,Как можно приобрести такую шубу либо заказать,0
241,И цену скажите пожалуйста,2


In [15]:
comments.drop(index=comments[comments['label']==3].index, inplace=True)

In [16]:
def lower_text(questions):
    
    clean_text=[]
    for question in questions:
        question=question.lower()
        clean_text.append(question)
    return clean_text

In [17]:
comments['clear_text']=lower_text(np.array(comments['question']))

In [18]:
def change(text):
    clean_text=[]
    for t in text:
        t=t.replace('й','и')
        t=t.replace('ё','е')
        clean_text.append(t)
    return clean_text
  


In [19]:
comments['clear_text']=change(np.array(comments['clear_text']))

In [20]:
def punctuation(text):
    clean_text=[]
    for t in text:
        t=re.sub(r'[^\w\s]',' ',t)
        clean_text.append(t)
    return clean_text

In [21]:
comments['clear_text']=punctuation(np.array(comments['clear_text']))

In [22]:
def size(text):
    clean_text=[]
    for t in text:
        t=re.sub(r'\d+(?:.\d+)?','цифра',t)
        
        t=re.sub(r'[a-z]+(?:.[a-z]+)?','буква',t)
        clean_text.append(t)
    return clean_text

In [23]:
comments['clear_text']=size(np.array(comments['clear_text']))

In [24]:
def spec_punctuation(text):
    clean_text=[]
    for t in text:
        t=re.sub(r'[^а-яА-Я0-9]',' ',t)
        clean_text.append(t)
    return clean_text

In [25]:
comments['clear_text']=spec_punctuation(np.array(comments['clear_text']))

In [26]:
def one(text):
    clean_text=[]
    for t in text:
        t=' '.join([w for w in t.split() if len(w)>2])
        clean_text.append(t)
    return clean_text

In [27]:
comments['clear_text']=one(np.array(comments['clear_text']))

In [28]:
comments[60:68]

,question,label,clear_text
60,45 46 есть в наличии?,1,цифра есть наличии
61,А как заказать,0,как заказать
62,Xs есть?,1,буква есть
63,Размеры M/S есть?,1,размеры буква есть
64,А размеры есть большие?,1,размеры есть большие
65,А есть 46-47? Хоть какие нибудь),1,есть цифра хоть какие нибудь
66,Есть 46 размер ?,1,есть цифра размер
67,Какие размеры?,1,какие размеры


In [29]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
from nltk import tokenize as tknz
from nltk.corpus import stopwords

In [31]:
def word_tokens(text):
    clean_text=[]
    for t in text:
        t=tknz.word_tokenize(t)
        clean_text.append(t)
    return clean_text

In [32]:
comments['tokens']=word_tokens(np.array(comments['clear_text']))

In [33]:
def stop_word_tokens(text):
    clean_tetx=[]
    stop_words = set(stopwords.words("russian"))
    for t in text:
        t=[word for word  in t if not word in stop_words]
        
        clean_tetx.append(t)
    return clean_tetx

In [34]:
comments['clean_tokens']=stop_word_tokens(np.array(comments['tokens']))

In [35]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 8.2MB 21.1MB/s 
     |████████████████████████████████| 378kB 58.2MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=849658 sha256=4fe04fad4f7c8aeef53e46af44cf78c0beecab73217084e5571d535c58a7f244
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [36]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [37]:
def stemmed_word_tokens(text):
    clean_text=[]
    
    for t in text:
        t=[morph.parse(word)[0].normal_form for word  in t]
        
        clean_text.append(t)
    return clean_text

In [38]:
comments['stem_tokens']=stemmed_word_tokens(np.array(comments['clean_tokens']))

In [40]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(comments['stem_tokens'], comments['label'])

In [41]:
tfidf = TfidfVectorizer(preprocessor=' '.join, min_df=3)
tfidf.fit(comments['stem_tokens'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 1), norm='l2',
                preprocessor=<built-in method join of str object at 0x7fe737fe0570>,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [43]:
xtrain_tfidf =  tfidf.transform(train_x).toarray()
xvalid_tfidf =  tfidf.transform(valid_x).toarray()

In [45]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [46]:
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, Count Vectors: ", accuracy)

LR, Count Vectors:  0.9272727272727272


In [47]:
classifier=linear_model.LogisticRegression()
classifier.fit(xtrain_tfidf, train_y)
predictions = classifier.predict(xvalid_tfidf)
metrics.accuracy_score(predictions, valid_y)

0.9272727272727272

Сверточные сети

In [51]:
max_words = 1000
max_len = 50
num_classes = 3

# Training
epochs = 20
batch_size = 5
print_batch_n = 5

In [52]:
num_classes = 3
y = keras.utils.to_categorical(comments["label"], num_classes)

In [53]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(comments['stem_tokens'], y)

In [54]:
xtrain_tfidf =  tfidf.transform(train_x).toarray()
xvalid_tfidf =  tfidf.transform(valid_x).toarray()

In [55]:
xtrain_tfidf[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.67145768, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.7410429 ])

In [56]:
l=[]
for i,values in comments['stem_tokens'].iteritems():
  for v in values:
    l.append(v)

  

s=comments['stem_tokens'].apply(' '.join)
max_len=len(set(l))



In [57]:
model = Sequential()
model.add(Embedding(input_dim=max_len+1, output_dim=32, input_length=41
                   ))
model.add(Conv1D(32, 2))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [59]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(xtrain_tfidf, train_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
 1/29 [>.............................] - ETA: 0s - loss: 1.0987 - accuracy: 0.6000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
29/29 [==============================] - 0s 14ms/step - loss: 1.0750 - accuracy: 0.4414 - val_loss: 1.0584 - val_accuracy: 0.4706
Epoch 2/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0445 - accuracy: 0.4414 - val_loss: 1.0381 - val_accuracy: 0.4706
Epoch 3/20
29/29 [==============================] - 0s 4ms/step - loss: 1.0310 - accuracy: 0.4414 - val_loss: 1.0371 - val_accuracy: 0.4706
Epoch 4/20
29/29 [==============================] - 0s 3ms/step - loss: 1.0255 - accuracy: 0.4414 - val_loss: 1.0399 - val_accuracy: 0.4706


Обработка вопроса пользователя

In [76]:
stop_words = set(stopwords.words("russian"))
def preprocess(t):
  t=t.lower()
  #t=t.replace('й','и')
  #t=t.replace('ё','е')
  t=re.sub(r'[^\w\s]',' ',t)
  t=re.sub(r'\d+(?:.\d+)?','цифра',t)
  t=re.sub(r'[a-z]+(?:.[a-z]+)?','буква',t)
  t=tknz.word_tokenize(t)
  t=[word for word  in t if (not word in stop_words) and (len(word)>2)]
  t=[morph.parse(word)[0].normal_form for word  in t]
  
  return t


In [65]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 409kB 12.8MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 2.6MB 55.8MB/s 


In [66]:

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [67]:
updater = Updater(token='1415308089:AAE0trR--jknLB0XLDLDD-3RrX5Cai_8q2s') # Токен API к Telegram
dispatcher = updater.dispatcher

In [68]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text='Здравствуйте, какой вопрос вас интересует?')

In [69]:
def textMessage(bot, update):
    response = 'Ваше сообщение принял ' + update.message.text # формируем текст ответа
    bot.send_message(chat_id=update.message.chat_id, text=response)

In [70]:
import logging
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

In [71]:
ship='Доставка возможна во все регионы России через службу доставки Boxberry. \
  Стоимость доставки рассчитывается при оформлении заказа. \
  Отследить отправление: https://boxberry.ru/tracking/\
  MEDIA LAND STORE принимает заказы в режиме online КРУГЛОСУТОЧНО. \
  Заказы обрабатываются по мере их поступления, по будням с 11:00 до 20:00.\
   Статус всех заказов обновляется автоматически.\
  К сожалению, на данный момент доставка в Украину не осуществляется'


In [72]:
price='Актуальные цены с учетом скидок и акций на весь ассортимент \
 товаров вы можете посомтреть на сайте https://medialand.store/nki/tproduct '


In [73]:
size='Наличие размера вы можете посмотреть в информации о товаре на сайте \
https://medialand.store/nki/tproduct. Если при выборе размера на сайте строка\
 с требуемым размером не активна, значит товара нет в наличии.'


In [74]:

  
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Здравствуйте, какой вопрос вас интересует?!')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    #print(txt)
    if np.amax(classifier.predict_proba(tfidf.transform([preprocess(txt)]).toarray()))>0.7:
      answer=classifier.predict(tfidf.transform([preprocess(txt)]).toarray())[0]
      #print(answer)
      if answer==0:
        update.message.reply_text(ship)
      elif answer==1:
        update.message.reply_text(size)
      elif answer==2:
        update.message.reply_text(price)
    else:
      
      
      update.message.reply_text('Переформулируйте вопрос, пожалуйста!')


   

In [75]:
updater = Updater("1415308089:AAE0trR--jknLB0XLDLDD-3RrX5Cai_8q2s", use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

2020-11-16 20:35:23,800 - apscheduler.scheduler - INFO - Scheduler started
2020-11-16 20:37:31,050 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2020-11-16 20:37:31,052 - apscheduler.scheduler - INFO - Scheduler has been shut down
